# Breast cancer detection from thermal imaging

The main purpose of this project is to develop a comprehensive decision support system for breast cancer screening.

## Library import
En esta sección se tratará de importar las librerias que se van a utilizar a lo largo de este modelo. Hay que tener en cuenta que parte de las librerías que utiliza este programa se encuentran declaradas en los ficheros que se encuentran en `src/scripts/*.py`.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from scripts import *

In [ ]:
computer.check_available_devices() # Check available devices

## Data selection
To make this model work correctly it will be necessary to extract and save the images found in the `data` folder.

In this folder there are two labeled folders that contain all the images to be used:
```
data
├── healthy
└── sick
```

In [ ]:
data = Data("./data/") # Data imported into a table

data.images.head(3) # Display first 3 rows

## Transformation
In the transformation stage, the data is adapted to find the solution to the problem to be solved.

First of all, the data obtained previously will be divided to be able to use it for training and to check the results.

In [ ]:
data.training, data.test = data.train_test_split(test_size=0.15, shuffle=True, stratify=True) # Split data into train and test

In [ ]:
# The category distribution is shown for the original, training, and test data
data.count_labels(data.images, "Original")
data.count_labels(data.training, "Training")
data.count_labels(data.test, "Test")

Once the data is divided, different transformation techniques are applied on it to expand the size of the dataset in real time while training the model.

In [ ]:
train_generator, validation_generator, test_generator = data.image_generator(shuffle=False) # Image genearation

In [ ]:
filters = {
	"original": lambda x: x,
	"red": lambda x: data.getImageTensor(x, (330, 0, 0), (360, 255, 255)) + data.getImageTensor(x, (0, 0, 0), (50, 255, 255)),
	"green": lambda x: data.getImageTensor(x, (60, 0, 0), (130, 255, 255)),
	"blue": lambda x: data.getImageTensor(x, (180, 0, 0), (270, 255, 255)),
}

data.show_images(train_generator, filters, "Training") # Show some images from the training generator

## Data Mining
This section seeks to apply techniques that are capable of extracting useful patterns and then evaluate them.

### Model creation
The model to be used for the next training is created.

In [ ]:
red_model = Model("red", filter=filters["red"], new=True, summary=False, plot=False) # Red model creation
green_model = Model("green", filter=filters["green"], new=True, summary=False, plot=False) # Green model creation
blue_model = Model("blue", filter=filters["blue"], new=True, summary=False, plot=False) # Blue model creation

In [ ]:
red_model.compile() # Compile the red model
green_model.compile() # Compile the green model
blue_model.compile() # Compile the blue model

### Model training
The created model is trained indicating the times that are going to be used.

In [ ]:
red_model.fit(train_generator, validation_generator, epochs=600, verbose=False, plot=False) # Train the red model
green_model.fit(train_generator, validation_generator, epochs=600, verbose=False, plot=False) # Train the green model
blue_model.fit(train_generator, validation_generator, epochs=600, verbose=False, plot=False) # Train the blue model

### Model evaluation
The trained model is evaluated using the generators created before. In this case, the best weight matrix obtained in the training will be used.

In [ ]:
red_model.evaluate(test_generator, path=None) # Evaluate the red model
green_model.evaluate(test_generator, path=None) # Evaluate the green model
blue_model.evaluate(test_generator, path=None) # Evaluate the blue model

### Grad-CAM
An activation map of the predictions obtained by the convolutional network is displayed.

In [ ]:
join_models = Join(red_model, green_model, blue_model)

# The activation map is displayed
for index, image in data.test.iterrows():
	join_models.visualize_heatmap(image)